<a href="https://colab.research.google.com/github/daichira-gif/LLM_test/blob/main/AdvancedGraphAGENT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 0. セットアップ（依存パッケージ）
# ==========================================
!pip install -q pandas networkx sentence-transformers faiss-cpu pyvis langchain langchain-openai openai tiktoken
!pip install -q --upgrade langchain-community

# ==========================================
# 1. ライブラリのインポート
# ==========================================
import os, json, time, getpass, re, hashlib, pathlib
import pandas as pd
import networkx as nx
import numpy as np
import faiss
from collections import Counter
from sentence_transformers import SentenceTransformer
import pickle  # ← 追加

# LangChain
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain.tools import StructuredTool

os.environ.setdefault("OMP_NUM_THREADS", "8")
print("Imports OK")

# ==========================================
# 2. OpenAI APIキー入力 & 環境変数サニタイズ
# ==========================================
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

def _is_ascii(s: str) -> bool:
    try:
        s.encode("ascii"); return True
    except Exception:
        return False

def sanitize_openai_env():
    for var in ("OPENAI_PROJECT", "OPENAI_ORGANIZATION", "OPENAI_ORG_ID"):
        val = os.environ.get(var)
        if val and not _is_ascii(val):
            print(f"[WARN] {var} に非ASCIIを検出。今回無効化: {val!r}")
            del os.environ[var]
sanitize_openai_env()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.6 MB/s eta 0:00:00
Imports OK
Enter your OpenAI API key: ··········


In [ ]:
# ==========================================
# 3. データ読み込み
# ==========================================
print("\n左のファイルアイコンから 'nodes_from_real_data.csv' と 'edges_from_real_data.csv' をアップロードしてください。")
input("アップロードが完了したら Enter を押してください: ")

nodes_df = pd.read_csv("nodes_from_real_data.csv")
edges_df = pd.read_csv("edges_from_real_data.csv")

def fmt_float(x):
    return str(int(x)) if x not in ("",) and pd.notna(x) else ""
if "date" in nodes_df.columns:
    nodes_df["date"] = nodes_df["date"].apply(fmt_float)
if "fiscal_year" in nodes_df.columns:
    nodes_df["fiscal_year"] = nodes_df["fiscal_year"].apply(fmt_float)
print("CSV読み込みOK")


左のファイルアイコンから 'nodes_from_real_data.csv' と 'edges_from_real_data.csv' をアップロードしてください。
アップロードが完了したら Enter を押してください: 
CSV読み込みOK


### テスト　tool_Requestをルールベースで発動

In [ ]:
class HybridGraphRAG:
    """
    ハイブリッドGraphRAG:
    - ツールベースの構造化分析（会議・発言・人物に対する集計やランキング）
    - 埋め込み検索による意味検索
    を統合し、質問内容に応じて適切な経路で回答します。
    """

    # ==========================
    # 0) ユーティリティ
    # ==========================
    @staticmethod
    def _extract_top_n(text: str, default_n: int = 5) -> int:
        """質問文に含まれる数字（例: トップ3, 上位3, 3人, Top 3）を拾って top_n にする。見つからなければ既定値。"""
        digits = []
        for ch in str(text):
            if "0" <= ch <= "9":
                digits.append(ch)
        if digits:
            try:
                n = int("".join(digits))
                return max(1, min(50, n))  # 下限1, 上限は適当に安全側
            except Exception:
                pass
        return default_n

    @staticmethod
    def _extract_keyword_for_utterance_query(text: str) -> str:
        """
        「〇〇に関する発言」「〇〇についての発言」から〇〇を素朴に抽出。
        失敗したら空文字を返す。
        """
        s = str(text)
        if "に関する" in s:
            return s.split("に関する")[0].strip()
        if "について" in s:
            return s.split("について")[0].strip()
        if "に触れた" in s:
            return s.split("に触れた")[0].strip()
        return ""

    @staticmethod
    def _format_ranking(d: dict, top_n: int, value_label: str = "score") -> str:
        """{'名前': 数値}の辞書を上位N件のランキング文字列に整形。"""
        items = list(d.items())[:top_n]
        lines = []
        for i, (name, val) in enumerate(items, 1):
            lines.append(f"{i}. {name}: {val}")
        return "\n".join(lines) if lines else "該当データが見つかりませんでした。"

    # ==========================
    # 1) 初期化
    # ==========================
    def __init__(self, nodes_df: pd.DataFrame, edges_df: pd.DataFrame):
        print("Initializing HybridGraphRAG...")
        self.nodes_df = nodes_df.fillna("")
        self.edges_df = edges_df
        self.G = self._build_graph()

        # 人名 → ID
        self.person_name_to_id = {
            row['name']: row['node_id']
            for _, row in self.nodes_df[self.nodes_df['label'] == 'Person'].iterrows()
        }

        # LLM / Agent
        self.llm = ChatOpenAI(model="gpt-4o", temperature=0)
        self.tools = self._get_tools()
        self.agent_executor = self._create_agent_executor()

        # 埋め込み + FAISS
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.index, self.idx2id = self._build_faiss_index()
        print("HybridGraphRAG is ready.")

    # ==========================
    # 2) グラフ構築
    # ==========================
    def _build_graph(self):
        G = nx.MultiDiGraph()
        for _, row in self.nodes_df.iterrows():
            G.add_node(row['node_id'], **row.to_dict())
        for _, row in self.edges_df.iterrows():
            if row['source'] in G and row['target'] in G:
                G.add_edge(row['source'], row['target'], relation=row['relation'])
        return G

    # ==========================
    # 3) FAISSインデックス
    # ==========================
    def _build_faiss_index(self):
        """各ノードを自然文に整形し、埋め込み→FAISSに格納する（空時も安全に初期化）。"""
        docs, idx2id = [], {}
        for i, (_, row) in enumerate(self.nodes_df.iterrows()):
            label = row.get('label', '')
            if label == "Person":
                text = f"人物: {row.get('name','')} 所属: {row.get('organization','')} 役職: {row.get('position','')}"
            elif label == "Meeting":
                text = f"会議: {row.get('date','')} 開催"
            elif label == "Utterance":
                text = f"発言: {row.get('content','')}"
            else:
                text = str(dict(row))
            docs.append(text)
            idx2id[i] = row['node_id']

        # 空データ対策：次元だけは確定して Index を作る
        if len(docs) == 0:
            dim = self.embedder.encode([""], normalize_embeddings=True).astype("float32").shape[1]
            index = faiss.IndexFlatIP(dim)
            return index, {}

        embeddings = self.embedder.encode(docs, normalize_embeddings=True).astype("float32")
        index = faiss.IndexFlatIP(embeddings.shape[1])  # 内積 = コサイン相当（正規化済み）
        if embeddings.shape[0] > 0:
            index.add(embeddings)
        return index, idx2id

    # ==========================
    # 4) LangChainツール登録（description を必ず付与）
    # ==========================
    def _get_tools(self):
        return [
            StructuredTool.from_function(
                self.list_all_people,
                description="グラフに存在する全人物の名前リストを返します。"
            ),
            StructuredTool.from_function(
                self.get_meetings_by_person,
                description="指定人物が参加した会議の日付一覧を返します。"
            ),
            StructuredTool.from_function(
                self.get_attendees_of_meeting,
                description="指定日付の会議に参加した人物一覧を返します。"
            ),
            StructuredTool.from_function(
                self.find_co_attendees,
                description="指定人物と頻繁に同席する人物のランキングを返します。"
            ),
            StructuredTool.from_function(
                self.get_most_attended_meeting,
                description="出席者数が最も多い会議の日付と人数を返します。"
            ),
            StructuredTool.from_function(
                self.count_utterances_by_person,
                description="指定人物の発言総数を返します。"
            ),
            StructuredTool.from_function(
                self.calculate_top_people_by_utterances_per_meeting,
                description="会議あたり平均発言数が多い人物のランキングを返します。"
            ),
            StructuredTool.from_function(
                self.search_utterances_by_keyword,
                description="キーワードを含む発言を検索し、発言者と会議日付と共に返します。"
            ),
            StructuredTool.from_function(
                self.filter_utterances_by_date,
                description="指定期間（必要ならキーワード条件付き）で発言を抽出します。"
            ),
            StructuredTool.from_function(
                self.calculate_top_people_by_topic,
                description="特定キーワードを多く発言した人物のランキングを返します。"
            ),
            StructuredTool.from_function(
                self.calculate_person_network_centrality,
                description="人物間ネットワークの中心性（betweenness/eigenvector/degree）ランキングを返します。"
            ),
        ]

    def _create_agent_executor(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "あなたは議事録グラフ分析の専門家です。"
             "集計やランキングはまずツールを用いて正確に算出してください。"
             "以下のような問い（中心/中心性/中央性/ネットワーク/ランキング/トップ/最多/平均/回数/人数/上位/順位）は必ずツールを使ってください。"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])
        agent = create_openai_tools_agent(self.llm, self.tools, prompt)
        return AgentExecutor(agent=agent, tools=self.tools, verbose=False)

    # ==========================
    # 5) ツール関数
    # ==========================
    def list_all_people(self):
        """全人物名のリストを返す。"""
        return list(self.person_name_to_id.keys())

    def get_meetings_by_person(self, person_name: str):
        """指定人物が参加した会議の日付一覧を返す。"""
        person_id = self.person_name_to_id.get(person_name)
        if not person_id:
            return f"エラー: {person_name} が見つかりません"
        attended_meetings = set()
        for neighbor in nx.all_neighbors(self.G, person_id):
            if self.G.nodes[neighbor]['label'] == 'Meeting':
                attended_meetings.add(neighbor)
            elif self.G.nodes[neighbor]['label'] == 'Utterance':
                for m in nx.all_neighbors(self.G, neighbor):
                    if self.G.nodes[m]['label'] == 'Meeting':
                        attended_meetings.add(m)
        return [self.G.nodes[mid]['date'] for mid in attended_meetings if self.G.nodes[mid]['date']]

    def get_attendees_of_meeting(self, meeting_date: str):
        """指定日付の会議に参加した人物一覧を返す。"""
        meeting_id = None
        for nid, data in self.G.nodes(data=True):
            if data.get('label') == 'Meeting' and str(data.get('date', '')).startswith(meeting_date):
                meeting_id = nid
                break
        if not meeting_id:
            return f"エラー: {meeting_date}の会議が見つかりません"
        attendees = set()
        for neighbor in nx.all_neighbors(self.G, meeting_id):
            if self.G.nodes[neighbor]['label'] == 'Utterance':
                for p in nx.all_neighbors(self.G, neighbor):
                    if self.G.nodes[p]['label'] == 'Person':
                        attendees.add(self.G.nodes[p]['name'])
        return list(attendees)

    def find_co_attendees(self, person_name: str):
        """指定人物と頻繁に同席する人物の上位を返す。"""
        meetings = self.get_meetings_by_person(person_name)
        if isinstance(meetings, str):
            return {"error": meetings}
        co = Counter()
        for date in meetings:
            for attendee in self.get_attendees_of_meeting(str(date).split('.')[0]):
                if attendee != person_name:
                    co[attendee] += 1
        return dict(co.most_common(10))

    def get_most_attended_meeting(self):
        """出席者が最も多い会議の日付と人数を返す。"""
        counts = {}
        for n, d in self.G.nodes(data=True):
            if d.get('label') == 'Meeting':
                date = d.get('date')
                counts[date] = len(self.get_attendees_of_meeting(str(date).split('.')[0]))
        most_attended = max(counts.items(), key=lambda x: x[1]) if counts else ("", 0)
        return {"date": most_attended[0], "attendee_count": most_attended[1]}

    def count_utterances_by_person(self, person_name: str):
        """指定人物の発言総数を返す。"""
        person_id = self.person_name_to_id.get(person_name)
        if not person_id:
            return 0
        return sum(1 for n in self.G.successors(person_id) if self.G.nodes[n]['label'] == 'Utterance')

    def calculate_top_people_by_utterances_per_meeting(self, top_n: int = 5):
        """会議あたり平均発言数の多い人物ランキングを返す。"""
        stats = {}
        for name in self.list_all_people():
            utt = self.count_utterances_by_person(name)
            meetings = self.get_meetings_by_person(name)
            if isinstance(meetings, list) and meetings:
                stats[name] = round(utt / len(meetings), 2)
        return dict(sorted(stats.items(), key=lambda x: x[1], reverse=True)[:top_n])

    def search_utterances_by_keyword(self, keyword: str):
        """キーワードを含む発言を検索し、発言者と会議日付を付けて返す。"""
        results = []
        for nid, data in self.G.nodes(data=True):
            if data.get('label') == 'Utterance' and isinstance(data.get('content'), str) and keyword in data.get('content', ''):
                entry = {'utterance': data.get('content', '')}
                entry['persons'] = [self.G.nodes[p]['name']
                                    for p in nx.all_neighbors(self.G, nid)
                                    if self.G.nodes[p]['label'] == 'Person']
                for m in nx.all_neighbors(self.G, nid):
                    if self.G.nodes[m]['label'] == 'Meeting':
                        entry['meeting_date'] = self.G.nodes[m].get('date')
                results.append(entry)
        return results

    def filter_utterances_by_date(self, start_date: str, end_date: str, keyword: str = None):
        """期間（start_date～end_date）内で、任意のキーワードを含む発言のみ抽出して返す。"""
        results = []
        for nid, data in self.G.nodes(data=True):
            if data.get('label') == 'Utterance':
                meeting_date = None
                for m in nx.all_neighbors(self.G, nid):
                    if self.G.nodes[m]['label'] == 'Meeting':
                        meeting_date = self.G.nodes[m].get('date')
                if meeting_date and start_date <= meeting_date <= end_date:
                    if keyword is None or (isinstance(data.get('content'), str) and keyword in data.get('content', '')):
                        results.append({'utterance': data.get('content'), 'date': meeting_date})
        return results

    def calculate_top_people_by_topic(self, keyword: str, top_n: int = 5):
        """特定キーワードを多く発言した人物のランキングを返す。"""
        counts = Counter()
        for nid, data in self.G.nodes(data=True):
            if data.get('label') == 'Utterance' and isinstance(data.get('content'), str) and keyword in data.get('content', ''):
                for p in nx.all_neighbors(self.G, nid):
                    if self.G.nodes[p]['label'] == 'Person':
                        counts[self.G.nodes[p]['name']] += 1
        return dict(counts.most_common(top_n))

    def calculate_person_network_centrality(self, metric: str = "betweenness"):
        """人物間ネットワークの中心性（betweenness/eigenvector/degree）を計算し上位を返す。"""
        person_nodes = [n for n, d in self.G.nodes(data=True) if d.get('label') == 'Person']
        person_graph = nx.Graph(); person_graph.add_nodes_from(person_nodes)
        meeting_nodes = [n for n, d in self.G.nodes(data=True) if d.get('label') == 'Meeting']
        for mid in meeting_nodes:
            attendees = [p for p in nx.all_neighbors(self.G, mid) if self.G.nodes[p]['label'] == 'Person']
            for i, p1 in enumerate(attendees):
                for p2 in attendees[i+1:]:
                    if person_graph.has_edge(p1, p2):
                        person_graph[p1][p2]['weight'] += 1
                    else:
                        person_graph.add_edge(p1, p2, weight=1)
        if person_graph.number_of_nodes() == 0:
            return {"error": "中心性を計算できる人物ノードが見つかりません"}
        if metric == "betweenness":
            centrality = nx.betweenness_centrality(person_graph, weight='weight', normalized=True)
        elif metric == "eigenvector":
            centrality = nx.eigenvector_centrality(person_graph, weight='weight')
        elif metric == "degree":
            centrality = nx.degree_centrality(person_graph)
        else:
            return {"error": f"未知のmetric指定: {metric}"}
        by_name = {self.G.nodes[nid]['name']: round(s, 4) for nid, s in centrality.items()}
        return dict(sorted(by_name.items(), key=lambda x: x[1], reverse=True)[:10])

    # ==========================
    # 6) 意味検索
    # ==========================
    def semantic_search(self, query: str, top_k=5):
        """意味検索：クエリに近いノードIDを上位 top_k 件返す。"""
        if getattr(self.index, "ntotal", 0) == 0:
            return []
        q_emb = self.embedder.encode([query], normalize_embeddings=True)
        D, I = self.index.search(q_emb, min(top_k, self.index.ntotal))
        return [self.idx2id[i] for i in I[0] if i >= 0 and i in self.idx2id]

    # ==========================
    # 7) ルーター（ヒューリスティック + エージェントfallback）
    # ==========================
    def query(self, question: str):
        """
        まずヒューリスティックで既知の集計系をツール直呼び（確実にランキングを出す）。
        当てはまらない場合のみ、エージェントに委譲。
        内容系（引用/どんなこと 等）は意味検索も使用。
        """
        q = str(question)
        q_top_n = self._extract_top_n(q, default_n=3 if "中心" in q or "中心性" in q or "中央性" in q else 5)

        # --- ヒューリスティック: 中心性
        if ("中心" in q or "中心性" in q or "中央性" in q or "ネットワーク" in q):
            metric = "degree"
            if "媒介" in q or "betweenness" in q.lower():
                metric = "betweenness"
            elif "固有" in q or "eigen" in q.lower():
                metric = "eigenvector"
            res = self.calculate_person_network_centrality(metric=metric)
            if isinstance(res, dict) and "error" in res:
                return res["error"]
            return "【ネットワーク中心性ランキング】\n" + self._format_ranking(res, q_top_n, value_label="centrality")

        # --- ヒューリスティック: 会議あたり平均発言
        if ("会議あたり" in q or "平均発言" in q or "発言の平均" in q):
            res = self.calculate_top_people_by_utterances_per_meeting(top_n=q_top_n)
            return "【会議あたり平均発言数ランキング】\n" + self._format_ranking(res, q_top_n, value_label="avg_utterances")

        # --- ヒューリスティック: 最多出席
        if ("最も出席" in q or "最多の出席" in q or "出席者が多い" in q):
            res = self.get_most_attended_meeting()
            if isinstance(res, dict):
                return f"【最多出席会議】\n日付: {res.get('date','不明')} / 出席者数: {res.get('attendee_count',0)}"
            return str(res)

        # --- ヒューリスティック: 「〇〇に関する発言」→ キーワード検索+引用
        if "発言" in q and ("に関する" in q or "について" in q or "に触れた" in q):
            kw = self._extract_keyword_for_utterance_query(q)
            if kw:
                rows = self.search_utterances_by_keyword(kw)
                if not rows:
                    return f"【検索結果】「{kw}」に関する発言は見つかりませんでした。"
                # 上位数件を引用表示
                lines = [f"【キーワード】{kw}（上位{min(5, len(rows))}件）"]
                for r in rows[:5]:
                    who = "、".join(r.get("persons", [])) or "（不明）"
                    quote = str(r.get("utterance", ""))[:200]
                    date = r.get("meeting_date", "（日付不明）")
                    lines.append(f"- {who} @ {date}\n  『{quote}』")
                return "\n".join(lines)

        # --- 内容系（発言/議論/内容/どんなこと）→ 意味検索
        if any(k in q for k in ["発言", "議論", "内容", "どんなこと"]):
            node_ids = self.semantic_search(q)
            if not node_ids:
                return "【意味検索結果】該当データが見つかりませんでした。"
            context_lines = []
            for nid in node_ids:
                data = self.G.nodes[nid]
                label = data.get('label', '')
                name = data.get('name', '')
                snippet = str(data.get('content', ''))[:150]
                date = data.get('date', '')
                context_lines.append(f"NodeID:{nid} | {label} | {name} | {date} | 『{snippet}』")
            return "【意味検索結果】\n" + "\n".join(context_lines)

        # --- それ以外はエージェントに委譲（ツール有効）
        out = self.agent_executor.invoke({"input": q})
        return out["output"] if isinstance(out, dict) and "output" in out else str(out)


In [ ]:
# ==========================================
# 7. 実行（HybridGraphRAG 版）
# ==========================================
rag = HybridGraphRAG(nodes_df, edges_df)
print("HybridGraphRAG initialized.")

question = "会議における議論ネットワークの中心にいた人物トップ3を教えてください。"
print(f"\n質問: {question}")
answer = rag.query(question)
print("\n--- 最終回答 ---")
print(answer)

print("\n--- 別の質問 ---")
question_2 = "ESGに関する発言をした人物と、その発言内容をいくつか引用付きで教えてください。"
print(f"質問: {question_2}")
answer_2 = rag.query(question_2)
print("\n--- 最終回答 ---")
print(answer_2)


Initializing HybridGraphRAG...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HybridGraphRAG is ready.
HybridGraphRAG initialized.

質問: 会議における議論ネットワークの中心にいた人物トップ3を教えてください。

--- 最終回答 ---
【ネットワーク中心性ランキング】
1. 水口 剛: 1.0
2. 林 礼子: 1.0
3. 足達 英一郎: 0.987

--- 別の質問 ---
質問: ESGに関する発言をした人物と、その発言内容をいくつか引用付きで教えてください。

--- 最終回答 ---
【キーワード】ESG（上位5件）
- 三木 誠 @ 20220224
  『では、私から御説明させていただきます。日本取引所グループサステナビリティ推進部の三木と申します。どうぞよろしくお願いします。 本日、資料、概要は資料１で今配付させていただいております。こちらと、あとは、中間報告書を今取りまとめ中でございます。こちらは精査する必要がもう少しございまして、画面のみの投映となりますが、資料１と画面のみの報告書の２種類で説明させていただきたいと思います。 では、報告書１枚』
- 岸上 有沙 @ 20220224
  『皆様、今年もどうぞよろしくお願いいたします。 私も、まずＤａｙ１で少しずつ導入していくという全体像に関しましては賛同いたします。その上で、もしかするとＤａｙ１.５ぐらいに期待することに関してコメントします。恐らく最初のESG関連債券が発行された時点からかなり議論も、グリーンウオッシュに対する認識も高まっておりますので、適格性をどのように判断したかですとか、そういった考え方の開示も進んできているかと』
- 岸上 有沙 @ 20220224
  『ありがとうございます。皆様の御議論の内容、大変興味深く聞いておりました。高村先生はじめ、賛同する内容も多くございましたので、重複しないところで追加させていただければと思います。先ほどビジネスモデルへの言及があったと思いますが、評価側から見たビジネスモデルの課題も共有いたします。もちろん例外もあると思いますが、マクロ的な視点として、どうしても調査費用を少しでも下げることによって最終的な受益者へのコス』
- 吉高 まり @ 20220224
  『ありがとうございます。私も林様、藤井様がおっしゃっているＩＯＳＣＯをベースに考えていかれることが重

### Tool_UseとSemantic_Searchを動的に作動させる

#### データ読み込み＋キャッシュ

In [ ]:
# ==========================================
# 3. データ読み込み
# ==========================================
print("\n左のファイルアイコンから 'nodes_from_real_data.csv' と 'edges_from_real_data.csv' をアップロードしてください。")
input("アップロードが完了したら Enter を押してください: ")

nodes_df = pd.read_csv("nodes_from_real_data.csv")
edges_df = pd.read_csv("edges_from_real_data.csv")

def fmt_float(x):
    return str(int(x)) if x not in ("",) and pd.notna(x) else ""
if "date" in nodes_df.columns:
    nodes_df["date"] = nodes_df["date"].apply(fmt_float)
if "fiscal_year" in nodes_df.columns:
    nodes_df["fiscal_year"] = nodes_df["fiscal_year"].apply(fmt_float)
print("CSV読み込みOK")

# ==========================================
# 4. キャッシュ仕組み（pickle + FAISS）
# ==========================================
CACHE_DIR = "graph_cache"
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)

def _df_fingerprint(df: pd.DataFrame) -> str:
    b = df.to_csv(index=False).encode("utf-8")
    return hashlib.sha256(b).hexdigest()

def build_data_hash(nodes_df: pd.DataFrame, edges_df: pd.DataFrame) -> str:
    return hashlib.sha256((_df_fingerprint(nodes_df) + _df_fingerprint(edges_df)).encode("utf-8")).hexdigest()

def save_assets(cache_key: str, G: nx.MultiDiGraph, person_map: dict,
                faiss_index: faiss.Index, idx_to_nodeid: dict):
    base = os.path.join(CACHE_DIR, cache_key)
    pathlib.Path(base).mkdir(exist_ok=True)
    with open(os.path.join(base, "graph.pkl"), "wb") as f:
        pickle.dump(G, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(base, "person_map.json"), "w", encoding="utf-8") as f:
        json.dump(person_map, f, ensure_ascii=False)
    with open(os.path.join(base, "idx_to_nodeid.json"), "w", encoding="utf-8") as f:
        json.dump(idx_to_nodeid, f, ensure_ascii=False)
    faiss.write_index(faiss_index, os.path.join(base, "index.faiss"))
    with open(os.path.join(base, "READY"), "w") as f:
        f.write("ok")

def load_assets(cache_key: str):
    base = os.path.join(CACHE_DIR, cache_key)
    if not os.path.exists(os.path.join(base, "READY")):
        return None
    try:
        with open(os.path.join(base, "graph.pkl"), "rb") as f:
            G = pickle.load(f)
        with open(os.path.join(base, "person_map.json"), "r", encoding="utf-8") as f:
            person_map = json.load(f)
        with open(os.path.join(base, "idx_to_nodeid.json"), "r", encoding="utf-8") as f:
            raw_map = json.load(f)
        # ★ JSONで文字列化されたキーを int に戻す
        idx_to_nodeid = {int(k): v for k, v in raw_map.items()}

        index = faiss.read_index(os.path.join(base, "index.faiss"))
        return G, person_map, index, idx_to_nodeid
    except Exception as e:
        print(f"[WARN] キャッシュ読込に失敗。再計算します: {e}")
        return None


左のファイルアイコンから 'nodes_from_real_data.csv' と 'edges_from_real_data.csv' をアップロードしてください。
アップロードが完了したら Enter を押してください: 
CSV読み込みOK


In [ ]:



# ==========================================
# 5. ツールレジストリ
# ==========================================
class ToolRegistry:
    def __init__(self, llm, G):
        self.llm = llm
        self.G = G
        self.tools = {}
        self.metadata = {}

    def register(self, fn, name, description, generated=False):
        tool = StructuredTool.from_function(fn, name=name, description=description)
        self.tools[name] = tool
        self.metadata[name] = {"description": description, "generated": generated}

    def get_tools(self):
        return list(self.tools.values())

    # --- 追加: まずは決め打ちマッピングで既存ツールを即時返す ---
    def _rule_based_pick(self, request: str):
        import re
        q = str(request)

        # ネットワーク中心性
        if re.search(r"(中心|中心性|中央性|ネットワーク)", q):
            return self.tools.get("calculate_person_network_centrality")

        # 会議あたり平均発言数
        if re.search(r"(会議あたり|平均).*発言", q):
            return self.tools.get("calculate_top_people_by_utterances_per_meeting")

        # 出席者最多の会議
        if re.search(r"(出席者|参加者).*(最多|最も多い)", q):
            return self.tools.get("get_most_attended_meeting")

        # 同席ランキング
        if "同席" in q:
            return self.tools.get("find_co_attendees")

        # 会議の参加者
        if re.search(r"(会議).*(参加者|出席者)", q):
            return self.tools.get("get_attendees_of_meeting")

        # 人物が参加した会議一覧
        if re.search(r"(参加した).*(会議)", q):
            return self.tools.get("get_meetings_by_person")

        # 発言のキーワード検索/引用
        if re.search(r"(発言|引用|ESG|キーワード|抽出)", q):
            return self.tools.get("search_utterances_by_keyword")

        return None

    def ensure_tool(self, request):
        # 1) まずは決め打ちマッピング（最優先）
        picked = self._rule_based_pick(request)
        if picked is not None:
            return picked

        # 2) 既存ツールのカタログを提示して LLM に選ばせる
        import json, difflib
        catalog = [
            {"name": name, "description": meta["description"]}
            for name, meta in self.metadata.items()
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are a senior Python developer.\n"
             "We already have some tools you can call. Here is the catalog (JSON):\n"
             f"{json.dumps(catalog, ensure_ascii=False)}\n"
             'If one existing tool can handle the user request, output JSON: {"use_existing":"<tool_name>"}.\n'
             "Otherwise, create a new Python function that uses the networkx graph `G`, "
             "`nodes_df`, `edges_df`, `Counter`. Return strict JSON with keys:\n"
             '  function_name (str), description (str), code (str).\n'
             "The function must accept **kwargs and return JSON-serializable.\n"
             "Output JSON only, no extra text."),
            ("user", "{request}")
        ])

        try:
            resp = (prompt | self.llm).invoke({"request": request}).content
            spec = json.loads(resp)
        except Exception:
            spec = {}

        # 2-a) 既存ツールを指名してきたらそれを使う
        if isinstance(spec, dict) and spec.get("use_existing"):
            return self.tools.get(spec["use_existing"])

        # 2-b) 新規ツール生成を指示してきた場合は exec して登録
        if isinstance(spec, dict) and spec.get("code") and spec.get("function_name"):
            local_env = {
                "G": self.G,
                "nodes_df": nodes_df,
                "edges_df": edges_df,
                "Counter": Counter,
            }
            # 最低限あると嬉しい依存を注入（生成コードが使っても落ちないように）
            try:
                import networkx as nx  # noqa
                import pandas as pd    # noqa
                import re              # noqa
                local_env.update({"nx": nx, "pd": pd, "re": re})
            except Exception:
                pass

            try:
                exec(spec["code"], local_env)
                fn = local_env[spec["function_name"]]
                name = spec["function_name"]
                desc = spec.get("description", "generated tool")
                self.register(fn, name, desc, generated=True)
                return self.tools[name]
            except Exception as e:
                print(f"[ToolGen Error] {e}")

        # 3) それでも決まらなければ、「似ている名前の既存ツール」を類似度で選ぶ
        names = list(self.tools.keys())
        if names:
            best = difflib.get_close_matches(str(request), names, n=1)  # ← 1文字ずつでなく全文で
            if best:
                return self.tools.get(best[0])


        return None


# ==========================================
# 6. AdvancedGraphAgent（GPU切替 & streaming完全OFF）
# ==========================================
USE_CUDA_FOR_EMBEDDING = True  # GPU使わないなら False

class AdvancedGraphAgent:
    def __init__(self, nodes_df: pd.DataFrame, edges_df: pd.DataFrame, verbose=False, use_cache=True):
        self.verbose = verbose

        # ------- LLM 初期化 -------
        # streaming=False を指定した上で…
        base_llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=False)
        # ★ streaming を強制OFF：どんなラップがかかっても param 'stream' を False で固定
        self.llm = base_llm.bind(stream=False)  # ← これが今回のキモ

        # ------- GPU 切替 -------
        device = "cpu"
        if USE_CUDA_FOR_EMBEDDING:
            try:
                import torch
                if torch.cuda.is_available():
                    device = "cuda"
            except Exception:
                device = "cpu"
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2", device=device)
        print(f"[Embedder] device = {device}")

        self.nodes_df = nodes_df.fillna("")
        self.edges_df = edges_df

        # ------- キャッシュ -------
        cache_key = build_data_hash(self.nodes_df, self.edges_df)
        assets = load_assets(cache_key) if use_cache else None
        if assets:
            print(f"[cache] hit. key={cache_key[:8]}...")
            self.G, self.person_name_to_id, self.index, self.idx_to_nodeid = assets
        else:
            print(f"[cache] miss. building assets for key={cache_key[:8]}...")
            self.G = self._build_graph()
            self.person_name_to_id = {
                row['name']: row['node_id']
                for _, row in self.nodes_df[self.nodes_df['label'] == 'Person'].iterrows()
            }
            self.index, self.idx_to_nodeid = self._build_faiss_index()
            if use_cache:
                save_assets(cache_key, self.G, self.person_name_to_id, self.index, self.idx_to_nodeid)

        self.tool_registry = ToolRegistry(self.llm, self.G)
        self._register_base_tools()
        self.router_agent_executor = self._create_router_agent()
        self.fusion_llm = self.llm

        self.cache = {}
        self.metrics_log = []
        self.new_facts = []

        # --- グラフ根拠評価用の閾値と状態 ---
        self.sim_threshold = 0.35  # 0〜1（cos類似）。必要なら調整
        self.last_semantic_hits = []  # 直近の意味検索ヒット（score付き）
        self.used_graph = False       # 今回の回答がグラフ根拠に依拠したか
        self.provenance_node_ids = [] # 根拠として採用したノードID
        self.output_provenance = []   # モデルが回答中に宣言した根拠ノード

        # __init__ の末尾あたりに追加
        self.MAX_TOOL_ITEMS = 12          # ツール結果の最大件数
        self.MAX_SNIPPET_CHARS = 160      # 1件あたりの抜粋最大文字数
        self.MAX_TOOL_CHARS = 3500        # ツール全文の最大文字数（融合前にクリップ）
        self.MAX_SEMANTIC_CHARS = 2000    # 意味検索コンテキストの最大文字数

    def _clip(self, s, n):
        s = "" if s is None else str(s)
        return s if len(s) <= n else (s[:n] + " …[truncated]")


    # ----- 初期化補助 -----
    def _build_graph(self):
        G = nx.MultiDiGraph()
        for _, row in self.nodes_df.iterrows():
            G.add_node(row['node_id'], **row.to_dict())
        for _, row in self.edges_df.iterrows():
            if row['source'] in G and row['target'] in G:
                G.add_edge(row['source'], row['target'], relation=row['relation'])
        return G

    def _build_faiss_index(self):
        docs, idx_to_nodeid = [], {}
        for i, (_, row) in enumerate(self.nodes_df.iterrows()):
            label = row.get('label', '')
            if label == "Person":
                text = f"人物: {row.get('name','')} 所属: {row.get('organization','')} 役職: {row.get('position','')}"
            elif label == "Meeting":
                text = f"会議: {row.get('date','')} 開催"
            elif label == "Utterance":
                text = f"発言: {row.get('content','')}"
            else:
                text = ""
            docs.append(text)
            idx_to_nodeid[i] = row['node_id']

        # 0件対策：次元だけ確保して空Indexを返す
        if len(docs) == 0:
            dim = self.embedder.encode([""], normalize_embeddings=True).astype("float32").shape[1]
            index = faiss.IndexFlatIP(dim)
            return index, {}

        embeddings = self.embedder.encode(docs, normalize_embeddings=True).astype("float32")
        index = faiss.IndexFlatIP(embeddings.shape[1])
        if embeddings.shape[0] > 0:
            index.add(embeddings)
        return index, idx_to_nodeid


    def _register_base_tools(self):
        self.tool_registry.register(self.get_meetings_by_person, "get_meetings_by_person",
                                    "指定された人物が参加した会議日付の一覧を返します。")
        self.tool_registry.register(self.get_attendees_of_meeting, "get_attendees_of_meeting",
                                    "指定された日付の会議の出席者リストを返します。")
        self.tool_registry.register(self.find_co_attendees, "find_co_attendees",
                                    "指定された人物と最も頻繁に同席した人物ランキングを返します。")
        self.tool_registry.register(self.get_most_attended_meeting, "get_most_attended_meeting",
                                    "最も出席者の多かった会議の日付と人数を返します。")
        self.tool_registry.register(self.count_utterances_by_person, "count_utterances_by_person",
                                    "指定された人物の発言回数を返します。")
        self.tool_registry.register(self.calculate_person_network_centrality, "calculate_person_network_centrality",
                                    "共起ネットワーク中心性ランキングを返します。")
        self.tool_registry.register(self.calculate_top_people_by_utterances_per_meeting,
                                    "calculate_top_people_by_utterances_per_meeting",
                                    "会議あたりの平均発言数が多い人物のランキングを返します。")
        self.tool_registry.register(self.search_utterances_by_keyword, "search_utterances_by_keyword",
                                    "キーワード（例: ESG）を含む発言を検索し、発言者と会議日付を返します。")


    def _create_router_agent(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "あなたは問い合わせルーターです。ユーザー質問を解析し、"
             "必ず次の2キーを持つ **厳密なJSON文字列のみ** を返してください。"
             ' keys: strategy, tool_request. '
             ' strategy は "tool_only" | "semantic_only" | "hybrid" のいずれか。'
             " 余計な説明やコードブロック、前置きは一切不要。"
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])
        agent = create_openai_tools_agent(self.llm, [], prompt)
        return AgentExecutor(agent=agent, tools=[], verbose=self.verbose)

    # AdvancedGraphAgent._heuristic_strategy
    def _heuristic_strategy(self, q: str) -> str:
        import re
        q = str(q)
        # 集計・ランキングは確実にツールへ
        if re.search(r"(中心|中心性|中央性|ネットワーク|ランキング|トップ|最多|平均|回数|人数|上位|順位)", q):
            return "tool_only"
        # ★ 内容・引用は「ハイブリッド」に変更（意味検索 + ツール）
        if re.search(r"(発言|議論|内容|引用|どんなこと|要約|概要|ESG)", q, flags=re.IGNORECASE):
            return "hybrid"
        return "hybrid"

    # ----- メイン機能 -----
    def _extract_json(self, text: str):
        try:
            return json.loads(text)
        except Exception:
            pass
        m = re.search(r'\{(?:[^{}]|(?R))*\}', text, flags=re.DOTALL)
        if not m:
            raise ValueError("Router did not return JSON.")
        return json.loads(m.group(0))

    def query(self, question: str):
        import time, re, json
        if question in self.cache:
            if self.verbose: print("Cache hit.")
            return self.cache[question]

        # 毎クエリでツール根拠をリセット
        self._last_tool_provenance_nodes = []

        start = time.time()
        router_output = self.router_agent_executor.invoke({"input": question})
        raw = router_output.get("output", str(router_output))
        try:
            plan = self._extract_json(raw)
        except Exception:
            plan = {"strategy": "hybrid", "tool_request": question}

        # まず router の値を安全化
        strategy = plan.get("strategy", "hybrid")
        tool_request = plan.get("tool_request")
        if not tool_request:  # null / false / "" は質問本文で補完
            tool_request = question

        # ルーター出力をヒューリスティックで補強
        heuristic = self._heuristic_strategy(question)
        if heuristic != "hybrid":
            strategy = heuristic

        # もしルールベースで既存ツールが選べるのに strategy が semantic_only ならハイブリッドへ昇格
        maybe_tool = self.tool_registry._rule_based_pick(tool_request)
        if maybe_tool and strategy == "semantic_only":
            strategy = "hybrid"

        tool_result, semantic_result = "N/A", "N/A"
        if strategy in ["tool_only", "hybrid"]:
            tool_result = self._execute_tool(tool_request)
        if strategy in ["semantic_only", "hybrid"]:
            semantic_result = self._execute_semantic_search(tool_request)

        if self.verbose:
            print(f"[router] raw={raw}")
            print(f"[plan] strategy={strategy} tool_request={tool_request}")

        final_answer = self._fuse_and_generate(question, tool_result, semantic_result)
        elapsed = time.time() - start

        self.cache[question] = final_answer
        self.metrics_log.append({"question": question, "strategy": strategy, "time_sec": round(elapsed, 3)})
        self._extract_new_facts(final_answer)

        # PROVENANCE 抽出・保存
        m = re.search(r'PROVENANCE:\s*([^\n\r]*)', final_answer, flags=re.IGNORECASE)
        self.output_provenance = []
        if m:
            raw_prov = m.group(1).strip()
            if raw_prov and raw_prov.lower() != "none":
                self.output_provenance = [s.strip() for s in raw_prov.split(",") if s.strip()]

        strong_hits = [h for h in self.last_semantic_hits if h.get("score", 0.0) >= self.sim_threshold]
        self.provenance_node_ids = [str(h["node_id"]) for h in strong_hits[:8]]
        used_any_tool = hasattr(self, "last_tools_used") and any(
            name for name in getattr(self, "last_tools_used", []) if name != "semantic_search"
        )
        self.used_graph = bool(self.output_provenance) or bool(strong_hits) or used_any_tool

        return final_answer



    def _execute_tool(self, tool_query: str):
        """選ばれた単一ツールを起動し、LLMに渡しても安全なサイズに整形して返す。"""
        import ast, json
        self.last_tools_used = []

        if tool_query is None or not str(tool_query).strip():
            return "（ツール）クエリが空のためスキップしました。"

        tool = self.tool_registry.ensure_tool(tool_query)
        if tool is None:
            return "ツールの生成に失敗しました。"

        tools = [tool]
        tool_desc = "\n".join([f"- {t.name}: {t.description}" for t in tools])
        tool_names = ", ".join([t.name for t in tools])

        tool_prompt = ChatPromptTemplate.from_messages([
            ("system",
            "与えられた指示に最も適した単一のツールを使って結果を返してください。\n"
            "利用可能なツール:\n{tools}\n利用可能なツール名: {tool_names}"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]).partial(tools=tool_desc, tool_names=tool_names)

        try:
            tool_agent = create_openai_tools_agent(self.llm, tools, tool_prompt)
            executor = AgentExecutor(agent=tool_agent, tools=tools, verbose=self.verbose)
            raw = executor.invoke({"input": str(tool_query)})
            self.last_tools_used = [tool.name]
            res = raw["output"] if isinstance(raw, dict) and "output" in raw else str(raw)

            # もし search_utterances_by_keyword なら、リスト→簡潔な行に圧縮
            if tool.name == "search_utterances_by_keyword":
                parsed = None
                if isinstance(res, list):
                    parsed = res
                elif isinstance(res, str):
                    import re, json, ast
                    # ★ res から最初の配列リテラルだけを抜き出す
                    m = re.search(r'\[[\s\S]*?\]', res)
                    if m:
                        blob = m.group(0)
                        try:
                            parsed = json.loads(blob)
                        except Exception:
                            try:
                                parsed = ast.literal_eval(blob)
                            except Exception:
                                parsed = None
                if isinstance(parsed, list):
                    lines = []
                    for r in parsed[: self.MAX_TOOL_ITEMS]:
                        who = ", ".join(r.get("persons", [])[:3]) or "（発言者不明）"
                        date = r.get("meeting_date") or "（日付不明）"
                        utt = (r.get("utterance", "") or "")[: self.MAX_SNIPPET_CHARS]
                        lines.append(f"- {date} | {who}: 「{utt}」")
                    # ★ 余計な説明や重複を避け、リスト整形だけを返す
                    res = "\n".join(lines) if lines else "（該当なし）"

            # 最終的にLLMに渡す前にクリップ
            res = self._normalize_tool_text(res)
            return self._clip(res, self.MAX_TOOL_CHARS)

        except Exception as e:
            return f"ツール実行でエラーが発生しました: {e}"


    def list_all_people(self):
        """グラフに存在する全人物名のリストを返す。"""
        # 既存の person_name_to_id をそのまま利用
        return list(self.person_name_to_id.keys())



    def _execute_semantic_search(self, query, top_k=5):
        import numpy as np
        if query is None or not str(query).strip():
            self.last_semantic_hits = []
            return "（意味検索）クエリが空のためスキップしました。"

        ntotal = getattr(self.index, "ntotal", 0)
        if ntotal == 0:
            self.last_semantic_hits = []
            return "（意味検索）インデックスが空です。"
        top_k = min(top_k, ntotal)

        q_emb = self.embedder.encode([str(query)], normalize_embeddings=True).astype("float32")
        D, I = self.index.search(q_emb, top_k)

        hits, context_lines = [], []
        for score, idx in zip(D[0], I[0]):
            if isinstance(idx, (int, np.integer)) and idx >= 0 and int(idx) in self.idx_to_nodeid:
                idx = int(idx)
                nid = self.idx_to_nodeid[idx]
                data = self.G.nodes[nid]
                hits.append({
                    "faiss_idx": idx,
                    "node_id": nid,
                    "label": data.get('label', ''),
                    "name": data.get('name', ''),
                    "content": data.get('content', ''),
                    "score": float(score)
                })
                context_lines.append(
                    f"NodeID:{nid} | Label:{data.get('label','')} | "
                    f"Name:{data.get('name','')} | Snippet:{str(data.get('content',''))[:120]} | Score:{score:.3f}"
                )

        self.last_semantic_hits = hits
        return "\n".join(context_lines) if context_lines else "（意味検索）該当コンテキストが見つかりませんでした。"



    def _fuse_and_generate(self, question, tool_result, semantic_result):
        from langchain.schema import HumanMessage
        import re

        if "search_utterances_by_keyword" in (getattr(self, "last_tools_used", []) or []):
              # PROVENANCE を構成（件数は最大8件に制限）
              strong_hits = [h for h in self.last_semantic_hits if h.get("score", 0.0) >= self.sim_threshold]
              suggested_ids = [str(h["node_id"]) for h in strong_hits[:8]]
              tool_nodes = [str(n) for n in getattr(self, "_last_tool_provenance_nodes", [])]
              provenance_ids = list(dict.fromkeys(tool_nodes + suggested_ids))[:8]
              prov = ", ".join(provenance_ids) if provenance_ids else "none"

              body = "以下は、ESGに関する発言の抜粋です（最大5件、各100文字程度）。\n" + \
                      self._clip(self._normalize_tool_text(tool_result), self.MAX_TOOL_CHARS)
              return body + f"\n\nPROVENANCE: {prov}"



        # 強制クリップ
        tool_result = self._clip(tool_result, self.MAX_TOOL_CHARS)
        semantic_result = self._clip(semantic_result, self.MAX_SEMANTIC_CHARS)

        strong_hits = [h for h in self.last_semantic_hits if h.get("score", 0.0) >= self.sim_threshold]
        suggested_ids = [str(h["node_id"]) for h in strong_hits[:8]]
        tool_nodes = [str(n) for n in getattr(self, "_last_tool_provenance_nodes", [])]
        provenance_ids = list(dict.fromkeys(tool_nodes + suggested_ids))
        suggested_ids_str = ", ".join(provenance_ids) if provenance_ids else "none"

        fusion_prompt = f"""あなたは高度な分析アシスタントです。以下の結果を統合し、元の質問に簡潔に答えてください。
    可能であれば、下の【候補根拠ノード】と【ツール結果】の内容を根拠として使ってください。

    ---
    【元の質問】
    {question}

    ---
    【ツール結果】
    {tool_result}

    ---
    【意味検索コンテキスト】
    {semantic_result}

    ---
    【候補根拠ノード】
    {suggested_ids_str}

    ---
    【出力要件】
    1) 簡潔で分かりやすい最終回答を先に書く
    2) 最後に1行だけ、実際に根拠として用いたノードIDを「PROVENANCE: id1,id2,...」形式で必ず出力する（無ければ "PROVENANCE: none"）
    """
        resp = self.fusion_llm.invoke([HumanMessage(content=fusion_prompt)])
        text = (resp.content or "").strip()

        # ★ LLMが書いた PROVENANCE を強制置換
        prov_line = f"PROVENANCE: {suggested_ids_str}"
        if re.search(r"(?im)^PROVENANCE:", text):
            text = re.sub(r"(?im)^PROVENANCE:\s*.*$", prov_line, text)
        else:
            text = text + "\n\n" + prov_line
        return text



    def _extract_new_facts(self, text):
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Extract factual triples (subject, predicate, object) from the text. Return as JSON list. If none, return []."),
            ("user", "{input}")
        ])
        try:
            resp = (prompt | self.llm).invoke({"input": text}).content
            triples = json.loads(resp)
            if isinstance(triples, list):
                self.new_facts.extend(triples)
        except Exception:
            pass

    def search_utterances_by_keyword(self, keyword: str, limit: int = None, snippet_chars: int = None):
        """キーワードを含む発言を検索し、発言者・会議日付とともに返す（件数/抜粋は上限を設ける）。"""
        import unicodedata
        limit = limit or self.MAX_TOOL_ITEMS
        snippet_chars = snippet_chars or self.MAX_SNIPPET_CHARS

        kw = unicodedata.normalize("NFKC", str(keyword)).lower()
        results = []
        for nid, data in self.G.nodes(data=True):
            if data.get('label') == 'Utterance' and isinstance(data.get('content'), str):
                text_norm = unicodedata.normalize("NFKC", data.get('content','')).lower()
                if kw in text_norm:
                    snippet = data.get('content','')[:snippet_chars]
                    entry = {'utterance': snippet}

                    # 発言者（前後両方向）
                    speakers = []
                    for p in list(self.G.predecessors(nid)) + list(self.G.successors(nid)):
                        if self.G.nodes[p].get('label') == 'Person':
                            speakers.append(self.G.nodes[p].get('name',''))
                    entry['persons'] = list({s for s in speakers if s})

                    # 会議日付（前後両方向）
                    meeting_date = None
                    for m in list(self.G.predecessors(nid)) + list(self.G.successors(nid)):
                        if self.G.nodes[m].get('label') == 'Meeting':
                            meeting_date = self.G.nodes[m].get('date')
                            break
                    entry['meeting_date'] = meeting_date

                    # 根拠ID（融合で使う）
                    self._last_tool_provenance_nodes.append(nid)

                    results.append(entry)
                    if len(results) >= limit:
                        break
        return results

    def _normalize_tool_text(self, text: str) -> str:
        s = "" if text is None else str(text)
        # LLM が勝手に足しがちな見出し行を除去（「以下は」で始まる行）
        kill_prefixes = ("以下は、ESGに関する発言", "以下は、ESGに関する発言をした")
        lines = []
        for ln in s.splitlines():
            t = ln.strip()
            if any(t.startswith(p) for p in kill_prefixes):
                continue
            lines.append(ln)
        # 連続空行の圧縮
        cleaned = []
        for ln in lines:
            if cleaned and not ln.strip() and not cleaned[-1].strip():
                continue
            cleaned.append(ln)
        return "\n".join(cleaned).strip()



    # ----- 既存ツール実装 -----
    def get_meetings_by_person(self, person_name: str):
        person_id = self.person_name_to_id.get(person_name)
        if not person_id:
            return f"エラー: 人物 '{person_name}' が見つかりません。"
        attended_meetings = set()
        for utterance_node in self.G.successors(person_id):
            if self.G.nodes[utterance_node].get('label') == 'Utterance':
                for meeting_node in self.G.successors(utterance_node):
                    if self.G.nodes[meeting_node].get('label') == 'Meeting':
                        attended_meetings.add(self.G.nodes[meeting_node].get('date'))
        return sorted([str(d) for d in attended_meetings if d])

    def get_attendees_of_meeting(self, meeting_date: str):
        meeting_id = next((nid for nid, data in self.G.nodes(data=True)
                           if str(data.get('date', '')).startswith(meeting_date)), None)
        if not meeting_id:
            return f"エラー: {meeting_date} の会議が見つかりません。"
        attendees = set()
        for utterance_node in self.G.predecessors(meeting_id):
            if self.G.nodes[utterance_node].get('label') == 'Utterance':
                for person_node in self.G.predecessors(utterance_node):
                    if self.G.nodes[person_node].get('label') == 'Person':
                        attendees.add(self.G.nodes[person_node]['name'])
        return list(attendees)

    def find_co_attendees(self, person_name: str):
        meetings_dates = self.get_meetings_by_person(person_name)
        if isinstance(meetings_dates, str):
            return {"error": meetings_dates}
        co_attendee_counts = Counter()
        for date in meetings_dates:
            attendees = self.get_attendees_of_meeting(str(date).split('.')[0])
            for attendee in attendees:
                if attendee != person_name:
                    co_attendee_counts[attendee] += 1
        return dict(co_attendee_counts.most_common(10))

    def get_most_attended_meeting(self):
        meeting_attendee_counts = {}
        for n, d in self.G.nodes(data=True):
            if d.get('label') == 'Meeting':
                date = str(d.get('date', '')).split('.')[0]
                if date:
                    attendees = self.get_attendees_of_meeting(date)
                    meeting_attendee_counts[date] = len(attendees)
        if not meeting_attendee_counts:
            return {"error": "会議が見つかりません。"}
        most_attended = max(meeting_attendee_counts.items(), key=lambda x: x[1])
        return {"date": most_attended[0], "attendee_count": most_attended[1]}

    def count_utterances_by_person(self, person_name: str):
        person_id = self.person_name_to_id.get(person_name)
        if not person_id:
            return f"エラー: 人物 '{person_name}' が見つかりません。"
        return sum(1 for n in self.G.successors(person_id) if self.G.nodes[n]['label'] == 'Utterance')

    def calculate_person_network_centrality(self, metric: str = "degree"):
        person_nodes = [n for n, d in self.G.nodes(data=True) if d.get('label') == 'Person']
        person_graph = nx.Graph()
        person_graph.add_nodes_from(person_nodes)
        meeting_nodes = [n for n, d in self.G.nodes(data=True) if d.get('label') == 'Meeting']
        for meeting_id in meeting_nodes:
            attendees = self.get_attendees_of_meeting(str(self.G.nodes[meeting_id].get('date', '')).split('.')[0])
            attendee_ids = [self.person_name_to_id.get(name) for name in attendees if self.person_name_to_id.get(name)]
            for i, p1 in enumerate(attendee_ids):
                for p2 in attendee_ids[i+1:]:
                    w = person_graph.get_edge_data(p1, p2, {'weight': 0})['weight'] + 1
                    person_graph.add_edge(p1, p2, weight=w)
        if person_graph.number_of_nodes() == 0:
            return {"error": "中心性を計算できる人物ノードが見つかりません。"}

        if metric == "betweenness":
            centrality = nx.betweenness_centrality(person_graph, weight='weight', normalized=True)
        elif metric == "eigenvector":
            centrality = nx.eigenvector_centrality(person_graph, weight='weight', max_iter=1000)
        else:
            centrality = nx.degree_centrality(person_graph)

        # 表示用ランキング
        by_name = {self.G.nodes[n]['name']: round(score, 4) for n, score in centrality.items()}
        top10 = dict(sorted(by_name.items(), key=lambda x: x[1], reverse=True)[:10])

        # ★ PROVENANCE: 上位者の Person ノードIDを覚える（例: 上位3名）
        name_to_id = {self.G.nodes[n]['name']: n for n, d in self.G.nodes(data=True) if d.get('label') == 'Person'}
        self._last_tool_provenance_nodes = [name_to_id[name] for name in list(top10.keys())[:3] if name in name_to_id]

        return top10


    def calculate_top_people_by_utterances_per_meeting(self, top_n: int = 5):
        """会議あたり平均発言数の多い人物ランキングを返す。"""
        stats = {}
        for name in self.list_all_people():
            utt = self.count_utterances_by_person(name)
            meetings = self.get_meetings_by_person(name)
            if isinstance(meetings, list) and meetings:
                stats[name] = round(utt / len(meetings), 2)
        return dict(sorted(stats.items(), key=lambda x: x[1], reverse=True)[:top_n])

# ==========================================
# 7. 実行
# ==========================================
agent = AdvancedGraphAgent(nodes_df, edges_df, verbose=True, use_cache=True)
print("AdvancedGraphAgent initialized.")

question = "会議における議論ネットワークの中心にいた人物トップ3を教えてください。"
print(f"\n質問: {question}")
answer = agent.query(question)
print("\n--- 最終回答 ---")
print(answer)

print("\n--- 別の質問 ---")
question_2 = "ESGに関する発言をした人物と、その発言内容をいくつか引用付きで教えてください。"
print(f"質問: {question_2}")
answer_2 = agent.query(question_2)
print("\n--- 最終回答 ---")
print(answer_2)

print("\n=== キャッシュ済みクエリ ===")
print(list(agent.cache.keys()))
print("\n=== メトリクスログ ===")
print(agent.metrics_log)
print("\n=== 新規抽出知識候補（トリプル） ===")
print(agent.new_facts)


[Embedder] device = cuda
[cache] hit. key=a65a39f3...
AdvancedGraphAgent initialized.

質問: 会議における議論ネットワークの中心にいた人物トップ3を教えてください。


> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":null}

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `calculate_person_network_centrality` with `{'metric': 'degree'}`


{'水口 剛': 1.0, '林 礼子': 1.0, '足達 英一郎': 0.987, '岸上 有沙': 0.961, '藤井 健司': 0.961, '手塚 宏之': 0.961, '長谷川 知子': 0.9351, '井口 譲二': 0.8701, '小野塚 惠美': 0.8312, '吉高 まり': 0.7922}会議における議論ネットワークの中心にいた人物トップ3は以下の通りです：

1. 水口 剛
2. 林 礼子
3. 足達 英一郎

> Finished chain.
[router] raw={"strategy":"semantic_only","tool_request":null}
[plan] strategy=tool_only tool_request=会議における議論ネットワークの中心にいた人物トップ3を教えてください。

--- 最終回答 ---
会議における議論ネットワークの中心にいた人物トップ3は、水口 剛、林 礼子、足達 英一郎です。

PROVENANCE: P_59485812, P_57473477, P_23590996

--- 別の質問 ---
質問: ESGに関する発言をした人物と、その発言内容をいくつか引用付きで教えてください。


> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":null}

> Finis

### test_results_from_graphrag.csvによる出力検証

In [ ]:
# ==========================================
# 9. CSV一括回答（最小パッチ版）
# ==========================================
import pandas as pd
from datetime import datetime

try:
    from zoneinfo import ZoneInfo
    _tz = ZoneInfo("Asia/Tokyo")
except Exception:
    _tz = None

# --- 直近ログフィールドの存在を保証 ---
if not hasattr(agent, "last_tools_used"):
    agent.last_tools_used = []
if not hasattr(agent, "last_strategy"):
    agent.last_strategy = ""

# --- モンキーパッチ（重複実行防止） ---
if not getattr(agent, "_batch_patch_done", False):
    _orig_query = agent.query
    _orig_execute_tool = agent._execute_tool
    _orig_execute_semantic_search = agent._execute_semantic_search

    def _patched_query(self, question: str):
        # 毎クエリごとにリセット
        self.last_tools_used = []
        # strategy は query 内で確定した時点で必ず更新されている前提
        return _orig_query(question)

    def _patched_execute_tool(self, tool_query):
        # ここで ensure_tool を呼び直すのは厳禁（重い＆副作用の恐れ）
        out = _orig_execute_tool(tool_query)
        # 元の _execute_tool が設定した last_tools_used を重複排除だけ
        self.last_tools_used = list(dict.fromkeys(self.last_tools_used))
        return out

    def _patched_execute_semantic_search(self, query, top_k=5):
        if "semantic_search" not in agent.last_tools_used:
            agent.last_tools_used.append("semantic_search")
        return _orig_execute_semantic_search(query, top_k=top_k)

    import types
    agent.query = types.MethodType(_patched_query, agent)
    agent._execute_tool = types.MethodType(_patched_execute_tool, agent)
    agent._execute_semantic_search = types.MethodType(_patched_execute_semantic_search, agent)
    agent._batch_patch_done = True

# === 入出力パス（コメントと実態を一致 & 元ファイルを残す）===
in_path = "/mnt/data/test_results_from_graphrag (2).csv"   # 実際にアップロードされたパスに合わせる
out_path = "/mnt/data/test_results_from_graphrag_answered.csv"

# 文字コードの揺れに備えるなら encoding 指定も可（UTF-8-BOMなど）
df = pd.read_csv(in_path)

# 必要列を用意（無ければ作る）
for col in ["質問", "回答", "ツール使用ログ", "処理時刻", "PROVENANCE", "根拠ノードIDs", "処理秒"]:
    if col not in df.columns:
        df[col] = ""

# 行ループ
for idx, row in df.iterrows():
    q = row.get("質問", "")
    if not isinstance(q, str) or not q.strip():
        continue
    try:
        ans = agent.query(q)

        # strategy は metrics_log の末尾から拾う（逐次実行前提）
        strategy = ""
        if agent.metrics_log:
            try:
                strategy = agent.metrics_log[-1].get("strategy", "")
                agent.last_strategy = strategy or agent.last_strategy
            except Exception:
                pass

        # ツール使用ログ（重複排除済みの last_tools_used を使用）
        tools_used = list(dict.fromkeys(agent.last_tools_used or []))
        tool_log = (agent.last_strategy or strategy or "unknown") + " | " + ",".join(tools_used)

        # PROVENANCE（回答本文から抽出済みの宣言）と、強ヒットの自動候補
        prov = getattr(agent, "output_provenance", []) or []
        prov_hits = getattr(agent, "provenance_node_ids", []) or []

        # 処理秒
        time_sec = ""
        if agent.metrics_log:
            try:
                time_sec = agent.metrics_log[-1].get("time_sec", "")
            except Exception:
                pass

        # 処理時刻（JST）
        now = datetime.now(_tz) if _tz else datetime.now()
        ts = now.strftime("%Y-%m-%d %H:%M:%S")

        # 書き込み
        df.at[idx, "回答"] = ans
        df.at[idx, "ツール使用ログ"] = tool_log
        df.at[idx, "処理時刻"] = ts
        df.at[idx, "PROVENANCE"] = ",".join(prov)
        df.at[idx, "根拠ノードIDs"] = ",".join(prov_hits)
        df.at[idx, "処理秒"] = time_sec

    except Exception as e:
        now = datetime.now(_tz) if _tz else datetime.now()
        df.at[idx, "回答"] = f"[ERROR] {e}"
        df.at[idx, "ツール使用ログ"] = "error"
        df.at[idx, "処理時刻"] = now.strftime("%Y-%m-%d %H:%M:%S")
        df.at[idx, "処理秒"] = ""

# 保存（元ファイルは残し、*_answered.csv として出力）
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"バッチ完了: {len(df)} 行を書き出しました → {out_path}")


In [ ]:
# # ==========================================
# # 9. CSV一括回答：質問→回答/ツール使用ログ/処理時刻 を埋める
# #    - 入力: /mnt/data/test_results_from_graphrag.csv
# #    - 出力: /mnt/data/test_results_from_graphrag_answered.csv
# # ==========================================
# import pandas as pd
# from datetime import datetime
# try:
#     # Python3.9+: zoneinfo（ColabはOK）
#     from zoneinfo import ZoneInfo
#     _tz = ZoneInfo("Asia/Tokyo")
# except Exception:
#     # フォールバック（タイムゾーン未対応環境）
#     _tz = None

# # --- 既存クラスに軽い拡張：直近のツール使用状況を保持 ---
# # （すでに定義済みならスキップされます）
# if not hasattr(agent, "last_tools_used"):
#     agent.last_tools_used = []
# if not hasattr(agent, "last_strategy"):
#     agent.last_strategy = ""

# # query() 開始時にリセット、戦略・意味検索使用の記録を追加
# # （一度だけモンキーパッチ。既存コードのロジックはそのまま）
# if not getattr(agent, "_batch_patch_done", False):
#     _orig_query = agent.query
#     _orig_execute_tool = agent._execute_tool
#     _orig_execute_semantic_search = agent._execute_semantic_search

#     def _patched_query(self, question: str):
#         # 直近状態をリセット
#         self.last_tools_used = []
#         self.last_strategy = ""
#         result = _orig_query(question)  # 既存の処理（内部で strategy を決める）
#         return result

#     def _patched_execute_tool(self, tool_query):
#         out = _orig_execute_tool(tool_query)
#         # ensure_tool で返されたツール名を記録（失敗時は記録しない）
#         # tool_query自体はログには長いので、ツール名だけにする
#         try:
#             tool = self.tool_registry.ensure_tool(tool_query)
#             if tool is not None:
#                 self.last_tools_used.append(tool.name)
#         except Exception:
#             pass
#         return out

#     def _patched_execute_semantic_search(self, query, top_k=5):
#         # 意味検索を使ったこと自体を記録
#         if "semantic_search" not in self.last_tools_used:
#             self.last_tools_used.append("semantic_search")
#         return _orig_execute_semantic_search(query, top_k=top_k)

#     # モンキーパッチ適用
#     import types
#     agent.query = types.MethodType(_patched_query, agent)
#     agent._execute_tool = types.MethodType(_patched_execute_tool, agent)
#     agent._execute_semantic_search = types.MethodType(_patched_execute_semantic_search, agent)

#     # 既存 query の中で決まった strategy を拾えるよう、_extract_json後～実行前の部分をフック
#     # ここは簡易に metrics_log の直近エントリから取得（なければ空）
#     # → 後続のバッチで最初に使うたびに metrics に積まれるため、回答後に拾う方式にします。
#     agent._batch_patch_done = True

# # === 入出力パス ===
# in_path = "/content/test_results_from_graphrag.csv"
# out_path = "/content/test_results_from_graphrag.csv"

# df = pd.read_csv(in_path)

# # 必要列が無ければ作成
# for col in ["質問", "回答", "ツール使用ログ", "処理時刻"]:
#     if col not in df.columns:
#         df[col] = ""

# # 行ループ
# for idx, row in df.iterrows():
#     q = row.get("質問", "")
#     if not isinstance(q, str) or not q.strip():
#         # 空行はスキップ
#         continue
#     try:
#         # 回答を取得
#         ans = agent.query(q)

#         # 直近の strategy を metrics_log の最後から推定（安全対策）
#         strategy = ""
#         if agent.metrics_log:
#             try:
#                 strategy = agent.metrics_log[-1].get("strategy", "")
#                 agent.last_strategy = strategy or agent.last_strategy
#             except Exception:
#                 pass

#         # ツール使用ログ文字列（例: "hybrid | semantic_search,get_meetings_by_person"）
#         tool_log = (agent.last_strategy or strategy or "unknown") + " | " + ",".join(agent.last_tools_used or [])

#         # 処理時刻（日本時間）
#         now = datetime.now(_tz) if _tz else datetime.now()
#         ts = now.strftime("%Y-%m-%d %H:%M:%S")

#         # 書き込み
#         df.at[idx, "回答"] = ans
#         df.at[idx, "ツール使用ログ"] = tool_log
#         df.at[idx, "処理時刻"] = ts

#     except Exception as e:
#         df.at[idx, "回答"] = f"[ERROR] {e}"
#         df.at[idx, "ツール使用ログ"] = "error"
#         now = datetime.now(_tz) if _tz else datetime.now()
#         df.at[idx, "処理時刻"] = now.strftime("%Y-%m-%d %H:%M:%S")

# # 保存（元ファイルは残し、*_answered.csv として出力）
# df.to_csv(out_path, index=False, encoding="utf-8-sig")
# print(f"バッチ完了: {len(df)} 行を書き出しました → {out_path}")




> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":false}

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_utterances_by_keyword` with `{'keyword': 'ESG', 'limit': 5}`


[{'utterance': 'よろしくお願いいたします。本日は、この第１回目のサステナブルファイナンス有識者会議にゲストとして呼んでいただきまして、大変ありがたく思っております。今ちょっとスクリーンを見渡してみますと、水口先生をはじめとして、私が去年の３月まで勤めておりましたＧＰＩＦ時代の私の活動の賛同者の方々がたくさん並んでいらっしゃいますので、', 'persons': ['水野 弘道'], 'meeting_date': '20210203'}, {'utterance': 'ありがとうございます。ただいま水口先生から御紹介にあずかりました、高村でございます。今日は、このサステナブルファイナンス、金融分野のそうそうたる専門家と実務家に囲まれて、大変緊張しております。水野さんには、２０１９年のパリ協定の長期成長戦略の過程の中でもいろいろ本当に重要な指摘をいただいて、あれが１つの跳躍台となって２', 'persons': ['高村 ゆかり'], 'meeting_date': '20210203'}, {'utterance': 'ありがとうございました。まさに大所高所からの御指摘をたくさんいただきました。全てに非常に感銘を受けているんですけれども、特に、１つは、途中でTragedy of the Horizonの話をいただきまして、やはりこういう金融システムの中に私たちはある種の規範といいましょうか、視点を組み込んでいくということが必要なんだな', 'persons': ['水口 剛'], 'meeting_date': '20210203'}, {'utterance': 'すみません、ちょっと別アカウントで入りました。田代さん、ありがとうございます。 まず国連のほうで何をやろうとしているかということにつ

/tmp/ipython-input-3692938018.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '金融庁のESG推進に向けた基本的スタンスは、サステナブルファイナンスの重要性を認識し、国際的な協力と長期的視点を重視していることです。特に、気候変動を含む広範なサステナビリティの視点を金融システムに組み込むことを目指しています。

PROVENANCE: none' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "回答"] = ans
/tmp/ipython-input-3692938018.py:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'hybrid | search_utterances_by_keyword,search_utterances_by_keyword,semantic_search' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "ツール使用ログ"] = tool_log
/tmp/ipython-input-3692938018.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2025-08-22 10:20:14' has dtype incompatible



> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":false}

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_utterances_by_keyword` with `{'keyword': 'ESG開示義務化', 'limit': 5}`


[]現在のところ、ESG開示義務化に関する具体的なロードマップに関する発言は見つかりませんでした。最新の情報については、関連する政府機関や規制当局の公式発表を確認することをお勧めします。

> Finished chain.
[router] raw={"strategy":"semantic_only","tool_request":false}
[plan] strategy=hybrid tool_request=企業に対するESG開示義務化のロードマップはどうなっていますか？


> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":false}

> Finished chain.
[router] raw={"strategy":"semantic_only","tool_request":false}
[plan] strategy=semantic_only tool_request=制度整備において想定されている課題は何ですか？


> Entering new AgentExecutor chain...
{"strategy":"semantic_only","tool_request":null}

> Finished chain.
[router] raw={"strategy":"semantic_only","tool_request":null}
[plan] strategy=semantic_only tool_request=政策によって企業行動がどう変化することを期待していますか？


> Entering new AgentExecutor chain.